## Description : Sentiment Analysis of Tweets fetched from Twitter using Python

In [ ]:
# Import required libraries
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
pd.set_option("display.max_columns", None)
plt.style.use('fivethirtyeight')


In [ ]:
# Load the API keys from the login credential file
log_cred = pd.read_csv("login_credentials.csv")
consumer_key = log_cred.iloc[0, 1]
consumer_secret = log_cred.iloc[1, 1]
access_token = log_cred.iloc[2, 1]
access_token_secret = log_cred.iloc[3, 1]


In [ ]:
# Create the authentication object and the API object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
username = input("Enter the user's screenname: ")
count = int(input("Enter the number of tweets to be retrieved: "))
tweets = api.user_timeline(screen_name=username, count=count,tweet_mode="extended")
# Not neccessary => Just to show that we have the tweets in the retrieved list
i = 1
for tweet in tweets[0:5]:
    print(f"{i}. {tweet.full_text}")
    i += 1

In [ ]:
# Create Dataframe of tweets
df = pd.DataFrame(data=[tweet.full_text for tweet in tweets], columns=['Tweets'])
df.shape

In [ ]:
# Clean Data - Remove special characters, links, and other elements
# Create a function to clean the data
def clean_tweet(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # Remove @mentions
    text = re.sub(r'#', '', text) # Remove the # in #hashtag
    text = re.sub(r'RT[\s]+', '', text) # Remove RT (retweet)
    text = re.sub(r'https?:\/\/\S+', '', text) # Remove hyperlinks

    return text

In [ ]:
# Apply the function to the dataframe
df['Tweets'] = df['Tweets'].apply(clean_tweet)
# Show cleaned tweets
df.head(10)

In [ ]:
# Create subjectivity and polarity then save to dataframe in new columns
def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def get_polarity(text):
    return TextBlob(text).sentiment.polarity

df['Subjectivity'] = df['Tweets'].apply(get_subjectivity)
df['Polarity'] = df['Tweets'].apply(get_polarity)
df.head(10)

In [ ]:
# Wordcloud of the top 100 most used words
all_words = ' '.join([text for text in df['Tweets']])

fig, ax = plt.subplots(figsize=(30, 40))
word_cloud = WordCloud(max_font_size=50, max_words=50, random_state=21).generate(all_words)

ax.imshow(word_cloud, interpolation='bilinear')
ax.set_title(f'WordCloud of the 20 Most Used Words in Recent Tweets by @{username}', fontsize=40)
ax.axis("off")

In [ ]:
# Compute negative, neutral, positive analysis of the tweets
def get_analysis(score):
    if score < 0:
        return 'Negative'
    elif score== 0:
        return 'Neutral'
    else:
        return 'Positive'

df['Analysis'] = df['Polarity'].apply(get_analysis)


In [ ]:
# Plotting the polarity of the tweets
ax = df['Analysis'].value_counts().plot(kind='bar', color=['green', 'blue', 'red'], figsize=(10, 8))
ax.set_title(f'Analysis of Tweets by @{username}', fontsize=10)

In [ ]:
def get_subjectivity_analysis(score):
    if score < 0.5:
        return 'Objective'
    else:
        return 'Subjective'

df['SUBJECTIVITY_ANALYSIS'] = df['Subjectivity'].apply(get_subjectivity_analysis)
df.head(10)

In [ ]:
# Plotting the polarity of the tweets
ax = df['SUBJECTIVITY_ANALYSIS'].value_counts().plot(kind='pie', autopct='%1.1f%%', figsize=(10, 8))
ax.set_title(f'Subjectivity Analysis of Tweets by @{username}', fontsize=10)

In [ ]:
# Print all of the positive tweets
positive_tweets = df[df['Analysis'] == 'Positive']
positive_tweets_count = positive_tweets['Tweets'].count()
print(positive_tweets['Tweets'])



In [ ]:
# Print top 10 the negative tweets
negative_tweets = df[df['Analysis'] == 'Negative']
negative_tweets_count = negative_tweets['Tweets'].count()
print(positive_tweets['Tweets'])

In [ ]:
# Print top the neutral tweets
neutral_tweets = df[df['Analysis'] == 'Neutral']
neutral_tweets_count = neutral_tweets['Tweets'].count()
print(neutral_tweets['Tweets'])

In [ ]:
# Pie chart of the polarity of the tweets
ax = df['Analysis'].value_counts().plot(kind='pie', autopct='%1.1f%%', figsize=(10, 8))
ax.set_title(f'% of Positive, Negative and Neutrral Tweets by @{username}', fontsize=10)


In [ ]:
# Plot polarity and subjectivity of the tweets
ax = df.plot(kind='scatter', x='Polarity', y='Subjectivity', color='blue', figsize=(16,8))
ax.title.set_text(f'Sentiment Analysis of @{username} Tweets')